# Explore Data

In [2]:
import pandas as pd
import os.path as osp

In [6]:
train_data_dir = '../data/train'
test_data_dir = '../data/test/test.csv'
topics = ['biology', 'cooking', 'crypto', 'diy', 'robotics', 'travel']

In [11]:
df_list = []
for t in topics:
    df_local = pd.read_csv(osp.join(train_data_dir, "".join([t, '.csv'])))
    df_local['topic'] = t
    df_list.append(df_local)
df_all = pd.concat(df_list)
del df_list

In [13]:
df_all.tail()

,id,title,content,tags,topic
19274,78013,Tipping in USA.California.SF,"<p>Being a <a href=""https://gsamaras.wordpress...",usa food-and-drink california tipping san-fran...,travel
19275,78016,"As a Canadian, what are appropriate gifts to b...","<p>I live in Vancouver, Canada and will be tra...",uk canada france culture gifts,travel
19276,78018,Does Macedonian police issue visitors a regist...,"<p>In Serbia and Macedonia, you have to Regist...",customs-and-immigration officials registration...,travel
19277,78019,Can my Austrian Visa D be renewed?,"<p>I am in Austria on a Visa D multiple entry,...",visas austria,travel
19278,78022,Shortest wide-body route from Frankfurt,<p>What is the shortest passenger route served...,untagged,travel


In [27]:
# randomly look into one row
def peek():
    df_slice = df_all.sample(1)
    print list(df_slice['title'])[0]
    print list(df_slice['content'])[0]
    print list(df_slice['tags'])[0]
    print list(df_slice['topic'])[0]

In [35]:
peek()

What happens if my ESTA expires on the day of arrival?
<p>I'm French and will travel the 14th of June to the US. My ESTA expires the 14th of  June 2013 and I'm wondering if my ESTA will be valid when I'll land in the US on the 14th of June 2013.</p>

<p>Does someone know anything about that?</p>

usa esta
travel
